In [1]:
import pandas as pd
from pathlib import Path

# --- 1. CONFIGURAÇÕES DE EXTRAÇÃO (EXTRACTION) ---
#    (VOCÊ PRECISA PREENCHER ISSO)
# ----------------------------------------------------

# TODO: Qual o nome EXATO do seu arquivo na pasta 01_raw_data?
NOME_ARQUIVO = 'AFASTADOS.xlsx' 

# TODO: Qual o NÚMERO da linha onde o cabeçalho REAL começa? 
# (Lembre-se: 0 = 1ª linha, 1 = 2ª linha, 2 = 3ª linha, etc.)
LINHA_CABECALHO = 2 # (Vou usar '2' como exemplo, baseado no que você disse)

# --- Caminhos (Não precisa mudar) ---
CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
    
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{NOME_ARQUIVO}' não encontrado em ../01_raw_data/")
    print("   Por favor, verifique o NOME_ARQUIVO e tente novamente.")
    # Pare a execução se o arquivo não for encontrado
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# --- 2. MAPEAMENTO DE COLUNAS (TRANSFORMATION) ---
#    (ESTA É A PARTE MAIS IMPORTANTE DO SEU TRABALHO AGORA)
# ----------------------------------------------------

# TODO: Mapeie os nomes da sua planilha (esquerda) para os nomes do banco (direita)
# Os nomes à DIREITA ('nome_completo', 'cpf', etc.) DEVEM ser iguais aos da sua tabela SQL.
COLUNAS_PARA_MANTER = {
    # 'NOME DA COLUNA NO EXCEL': 'nome_no_banco_sql',
    'NOME DO EMPREGADO': 'nome_completo',     
    'CPF': 'cpf',
    'DATA DE INICIO': 'data_admissao',
    'FUNÇÃO': 'cargo',
    'DATA DE RETORNO': 'data_demissao',    # Mapeando retorno para demissão por enquanto
    'SITUAÇÃO ATUAL': 'status_atual'       
    # 'ID_UNICO_FUNCIONARIO': 'funcionario_id' # Se você tiver um ID, é perfeito!
}

# Aplicando a renomeação
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)

# Selecionando APENAS as colunas que definimos
colunas_finais = list(COLUNAS_PARA_MANTER.values())

# Verificando se todas as colunas mapeadas existem
colunas_faltando = [col for col in colunas_finais if col not in df_limpo.columns]
if colunas_faltando:
     print(f"❌ ERRO DE MAPEAMENTO: As colunas brutas no dicionário não foram encontradas. Verifique a ortografia.")
     print(f"   Colunas esperadas (limpas): {colunas_finais}")
     print(f"   Colunas encontradas (brutas): {list(df.columns)}")
     raise ValueError("Erro no mapeamento de colunas.")

df_limpo = df_limpo[colunas_finais]

# Limpeza simples de dados
df_limpo.dropna(subset=['nome_completo'], inplace=True) # Remove linhas sem nome
df_limpo['status_atual'] = df_limpo['status_atual'].fillna('Ativo') # Assume 'Ativo' se estiver vazio

print("✅ Mapeamento e limpeza concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")
print("\n--- AMOSTRA DOS DADOS PRONTOS PARA CARGA (LOAD) ---")
print(df_limpo.head())

Iniciando ETL para o arquivo: AFASTADOS.xlsx
--- FASE 1: EXTRAÇÃO (READ) ---
✅ Arquivo lido com sucesso. Dimensões brutas: (126, 11)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---
❌ ERRO DE MAPEAMENTO: As colunas brutas no dicionário não foram encontradas. Verifique a ortografia.
   Colunas esperadas (limpas): ['nome_completo', 'cpf', 'data_admissao', 'cargo', 'data_demissao', 'status_atual']
   Colunas encontradas (brutas): ['Colaborador', 'Data da Admissão', 'Data do Atestado ', 'Dias Afastado', 'CID', 'Descrição', 'Doutor', 'CRM', 'Agendamento do INSS ', 'Aprovação do INSS', 'Afastado até ']


ValueError: Erro no mapeamento de colunas.

In [2]:
import pandas as pd
from pathlib import Path

# --- 1. CONFIGURAÇÕES DE EXTRAÇÃO (EXTRACTION) ---
NOME_ARQUIVO = 'AFASTADOS.xlsx' 
LINHA_CABECALHO = 2 # Vamos manter '2' (3ª linha), pois parece ter lido os cabeçalhos corretos.

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO

print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
    
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo '{NOME_ARQUIVO}' não encontrado em ../01_raw_data/")
    raise
except Exception as e:
    print(f"❌ ERRO AO LER O ARQUIVO (Verifique a LINHA_CABECALHO): {e}")
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# --- 2. MAPEAMENTO DE COLUNAS (A CORREÇÃO) ---
# Mapeando os nomes REAIS da sua planilha (esquerda) para os nomes do banco (direita)
# Note que pegamos SÓ o que precisamos para a 'dim_funcionario'
COLUNAS_PARA_MANTER = {
    # 'NOME DA COLUNA NO EXCEL': 'nome_no_banco_sql',
    'Colaborador': 'nome_completo',     
    'Data da Admissão': 'data_admissao'
    # 'Afastado até ': 'data_demissao' # Poderíamos usar isso, mas vamos focar no status
    # 'Descrição': 'status_atual'      # A 'Descrição' (Ex: "Atestado Médico") é um bom status
}

# Aplicando a renomeação
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)

# Selecionando APENAS as colunas que definimos
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())
df_limpo = df_limpo[colunas_finais_mapeadas]

# --- 3. CRIAÇÃO DE NOVAS COLUNAS (TRANSFORMATION) ---

# Se o arquivo se chama 'AFASTADOS.xlsx', podemos assumir que o status é 'Afastado'
df_limpo['status_atual'] = 'Afastado' 

# Adicionando as colunas que o banco SQL espera, mas que este arquivo não tem
# Elas ficarão vazias (None/NaN) por enquanto, esperando outra planilha
df_limpo['cpf'] = None
df_limpo['cargo'] = None
df_limpo['data_demissao'] = None
df_limpo['funcionario_id'] = None # Vamos precisar de um ID único no futuro

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL (dim_funcionario)
colunas_finais_sql = ['funcionario_id', 'nome_completo', 'cpf', 'data_admissao', 'cargo', 'data_demissao', 'status_atual']
df_limpo = df_limpo[colunas_finais_sql]

# Limpeza final
df_limpo.dropna(subset=['nome_completo'], inplace=True) # Remove linhas sem nome

print("✅ Mapeamento e criação de colunas concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")
print("\n--- AMOSTRA DOS DADOS PRONTOS PARA CARGA (LOAD) ---")
print(df_limpo.head())

Iniciando ETL para o arquivo: AFASTADOS.xlsx
--- FASE 1: EXTRAÇÃO (READ) ---
✅ Arquivo lido com sucesso. Dimensões brutas: (126, 11)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---
✅ Mapeamento e criação de colunas concluídos.
Dimensões após a limpeza: (65, 7)

--- AMOSTRA DOS DADOS PRONTOS PARA CARGA (LOAD) ---
   funcionario_id              nome_completo   cpf        data_admissao cargo  \
0            None     ADAILTON SILVA BARRETO  None  2013-10-24 00:00:00  None   
3            None                Colaborador  None     Data da Admissão  None   
4            None  EDICLELHO ROLAND ZAMBELLI  None  2013-03-08 00:00:00  None   
9            None                Colaborador  None     Data da Admissão  None   
10           None  RAIMUNDO DE MIRANDA SOUSA  None  2012-04-01 00:00:00  None   

   data_demissao status_atual  
0           None     Afastado  
3           None     Afastado  
4           None     Afastado  
9           None     Afastado  
10          None     Afastado  


In [3]:
from sqlalchemy import create_engine
import io # Usaremos esta biblioteca para otimizar a carga

print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (LOAD) ---
#    (VOCÊ PRECISA PREENCHER A SENHA)
# ----------------------------------------------------

# Lembre-se das nossas configurações do pgAdmin
DB_USER = 'postgres'
DB_PASS = 'Hserv@2025' # TODO: Coloque sua senha
DB_HOST = 'localhost'
DB_PORT = '5433' # A porta especial que descobrimos!
DB_NAME = 'hserv_dw' # O nome do Data Warehouse que criamos

# Criando a "string de conexão"
# É como o "endereço" completo do banco de dados
try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: Não foi possível conectar ao PostgreSQL.")
    print(f"   Verifique se sua SENHA, PORTA ({DB_PORT}) e NOME DO BANCO ({DB_NAME}) estão corretos.")
    print(f"   Erro original: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
# ----------------------------------------------------

NOME_TABELA_SQL = 'dim_funcionario'

try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    # O comando mágico do Pandas para enviar o DataFrame para o SQL
    # if_exists='append': Adiciona os novos dados. Se a tabela já tiver dados, ele não apaga os antigos.
    # index=False: Não salva o "índice" do Pandas (a coluna 0, 1, 2...) no banco.
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', # 'replace' apaga tudo, 'append' adiciona
        index=False,
        schema='public' # O schema padrão que criamos
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA: Não foi possível salvar os dados no banco.")
    print(f"   Verifique se os nomes das colunas no DataFrame (ex: 'nome_completo') batem EXATAMENTE")
    print(f"   com os nomes das colunas na sua tabela SQL no pgAdmin.")
    print(f"   Erro original: {e}")
    raise


--- FASE 3: CARGA (LOAD) ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 65 linhas na tabela 'dim_funcionario'...
❌ ERRO NA CARGA: Não foi possível salvar os dados no banco.
   Verifique se os nomes das colunas no DataFrame (ex: 'nome_completo') batem EXATAMENTE
   com os nomes das colunas na sua tabela SQL no pgAdmin.
   Erro original: (psycopg2.OperationalError) could not translate host name "2025@localhost" to address: Non-recoverable failure in name resolution

(Background on this error at: https://sqlalche.me/e/20/e3q8)


OperationalError: (psycopg2.OperationalError) could not translate host name "2025@localhost" to address: Non-recoverable failure in name resolution

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [4]:
from sqlalchemy import create_engine
import io
from urllib.parse import quote_plus # <-- IMPORTAMOS A FERRAMENTA DE CORREÇÃO

print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (CORRIGIDO) ---
# ----------------------------------------------------

DB_USER = 'postgres'
DB_HOST = 'localhost'
DB_PORT = '5433'
DB_NAME = 'hserv_dw'

# TODO: Coloque sua senha AQUI (com o @ ou o que for)
RAW_DB_PASS = 'Hserv@2025' 

# Codificando a senha para caracteres especiais (como @, #, $, etc.)
DB_PASS = quote_plus(RAW_DB_PASS)

# Criando a "string de conexão"
try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
# (Este bloco continua o mesmo)
# ----------------------------------------------------

NOME_TABELA_SQL = 'dim_funcionario'

try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA: {e}")
    raise


--- FASE 3: CARGA (LOAD) ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 65 linhas na tabela 'dim_funcionario'...
❌ ERRO NA CARGA: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [5]:
from sqlalchemy import create_engine
import io
from urllib.parse import quote_plus # <-- IMPORTAMOS A FERRAMENTA DE CORREÇÃO

print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (CORRIGIDO) ---
# ----------------------------------------------------

DB_USER = 'postgres'
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'hserv_dw'

# TODO: Coloque sua senha AQUI (com o @ ou o que for)
RAW_DB_PASS = 'SUA_SENHA_DO_POSTGRESQL_AQUI' 

# Codificando a senha para caracteres especiais (como @, #, $, etc.)
DB_PASS = quote_plus(RAW_DB_PASS)

# Criando a "string de conexão"
try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
# (Este bloco continua o mesmo)
# ----------------------------------------------------

NOME_TABELA_SQL = 'dim_funcionario'

try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA: {e}")
    raise


--- FASE 3: CARGA (LOAD) ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 65 linhas na tabela 'dim_funcionario'...
❌ ERRO NA CARGA: 'utf-8' codec can't decode byte 0xe7 in position 78: invalid continuation byte


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe7 in position 78: invalid continuation byte

In [6]:
from sqlalchemy import create_engine
import io
from urllib.parse import quote_plus

print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (CORRIGIDO NOVAMENTE) ---
# ----------------------------------------------------

DB_USER = 'postgres'
DB_HOST = 'localhost'
DB_PORT = '5433'
DB_NAME = 'hserv_dw'

# TODO: Coloque sua senha AQUI (com o @, ç, ou o que for)
RAW_DB_PASS = 'Hserv@2025' 

# Codificando a senha para caracteres especiais (como @, #, $, etc.)
DB_PASS = quote_plus(RAW_DB_PASS)

# Criando a "string de conexão"
try:
    # A MUDANÇA ESTÁ AQUI: Adicionamos ?client_encoding=utf8 no final
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
# (O restante do código é o mesmo)
# ----------------------------------------------------

NOME_TABELA_SQL = 'dim_funcionario'

try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA: {e}")
    raise


--- FASE 3: CARGA (LOAD) ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 65 linhas na tabela 'dim_funcionario'...
❌ ERRO NA CARGA: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)


OperationalError: (psycopg2.OperationalError) connection to server at "localhost" (::1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5433 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [7]:
from sqlalchemy import create_engine
import io
from urllib.parse import quote_plus # Para lidar com caracteres especiais (@, ç)

print("\n--- FASE 3: CARGA (LOAD) - TENTATIVA FINAL DE CONEXÃO ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (TODAS AS CORREÇÕES) ---
# ------------------------------------------------------------------

DB_USER = 'postgres'
DB_HOST = '127.0.0.1'     # Usando o IP local mais seguro
DB_PORT = '5432'          # Porta correta (confirmada pelo SHOW port;)
DB_NAME = 'hserv_dw'

# Sua senha é Hserv@2025
RAW_DB_PASS = 'Hserv@2025' 

# 1. Codificando a senha (para lidar com o '@')
DB_PASS = quote_plus(RAW_DB_PASS)

# 2. Criando a string de conexão (com o encoding para lidar com o 'ç')
try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: Não foi possível conectar ao PostgreSQL.")
    print(f"   Por favor, verifique se a senha '{RAW_DB_PASS}' está correta.")
    print(f"   Erro original: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
# -------------------------------------------------------------------

NOME_TABELA_SQL = 'dim_funcionario'

try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento das colunas): {e}")
    raise


--- FASE 3: CARGA (LOAD) - TENTATIVA FINAL DE CONEXÃO ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 65 linhas na tabela 'dim_funcionario'...
❌ ERRO NA CARGA (Verifique o mapeamento das colunas): (psycopg2.errors.InvalidDatetimeFormat) ERRO:  sintaxe de entrada é inválida para tipo date: "Data da Admissão"
LINE 1: ...L, NULL, 'Afastado'), (NULL, 'Colaborador', NULL, 'Data da A...
                                                             ^

[SQL: INSERT INTO public.dim_funcionario (funcionario_id, nome_completo, cpf, data_admissao, cargo, data_demissao, status_atual) VALUES (%(funcionario_id__0)s, %(nome_completo__0)s, %(cpf__0)s, %(data_admissao__0)s, %(cargo__0)s, %(data_demissao__0)s, %(st ... 9133 characters truncated ... 4)s, %(cpf__64)s, %(data_admissao__64)s, %(cargo__64)s, %(data_demissao__64)s, %(status_atual__64)s)]
[parameters: {'cpf__0': None, 'data_admissao__0': datetime.datetime(2013, 10, 24, 0, 0), 'data_demissao__0': None

DataError: (psycopg2.errors.InvalidDatetimeFormat) ERRO:  sintaxe de entrada é inválida para tipo date: "Data da Admissão"
LINE 1: ...L, NULL, 'Afastado'), (NULL, 'Colaborador', NULL, 'Data da A...
                                                             ^

[SQL: INSERT INTO public.dim_funcionario (funcionario_id, nome_completo, cpf, data_admissao, cargo, data_demissao, status_atual) VALUES (%(funcionario_id__0)s, %(nome_completo__0)s, %(cpf__0)s, %(data_admissao__0)s, %(cargo__0)s, %(data_demissao__0)s, %(st ... 9133 characters truncated ... 4)s, %(cpf__64)s, %(data_admissao__64)s, %(cargo__64)s, %(data_demissao__64)s, %(status_atual__64)s)]
[parameters: {'cpf__0': None, 'data_admissao__0': datetime.datetime(2013, 10, 24, 0, 0), 'data_demissao__0': None, 'nome_completo__0': 'ADAILTON SILVA BARRETO', 'cargo__0': None, 'status_atual__0': 'Afastado', 'funcionario_id__0': None, 'cpf__1': None, 'data_admissao__1': 'Data da Admissão', 'data_demissao__1': None, 'nome_completo__1': 'Colab

In [8]:
# --- 2. MAPEAMENTO DE COLUNAS (A CORREÇÃO DA SUJEIRA) ---
# ... (tudo igual aqui) ...

# Aplicando a renomeação (mantendo a mesma lógica da última vez)
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Selecionando APENAS as colunas que definimos
df_limpo = df_limpo[colunas_finais_mapeadas]

# --- 3. CRIAÇÃO DE NOVAS COLUNAS E LIMPEZA (AQUI ESTÁ A CORREÇÃO!) ---

# 3.1 Limpeza Final: REMOVENDO LINHAS LIXO
# Remove linhas onde o nome do colaborador NÃO é um nome de verdade
df_limpo = df_limpo[df_limpo['Colaborador'] != 'Colaborador'] # Remove a linha onde a coluna Colaborador é igual ao texto 'Colaborador'
df_limpo.dropna(subset=['Colaborador'], inplace=True) # Remove qualquer linha que ainda esteja vazia na coluna nome.

# 3.2 Criação e Reordenação das Colunas (O mesmo que antes)
df_limpo['status_atual'] = 'Afastado' 
df_limpo['cpf'] = None
df_limpo['cargo'] = None
df_limpo['data_demissao'] = None
df_limpo['funcionario_id'] = None 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL (dim_funcionario)
colunas_finais_sql = ['funcionario_id', 'nome_completo', 'cpf', 'data_admissao', 'cargo', 'data_demissao', 'status_atual']
df_limpo.columns = ['nome_completo', 'data_admissao'] # Precisamos renomear de volta após a limpeza para bater com a lista de colunas SQL
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento e criação de colunas concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")
print("\n--- AMOSTRA DOS DADOS PRONTOS PARA CARGA (LOAD) ---")
print(df_limpo.head())

KeyError: 'Colaborador'

In [9]:
# --- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) - CORREÇÃO DE KEYERROR ---

# 2. MAPEAMENTO DE COLUNAS (O mesmo da última vez)
COLUNAS_PARA_MANTER = {
    # Mapeando os nomes REAIS da sua planilha (esquerda) para os nomes do banco (direita)
    'Colaborador': 'nome_completo',     
    'Data da Admissão': 'data_admissao'
}

# Aplicando a renomeação
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())
df_limpo = df_limpo[colunas_finais_mapeadas]

# --- 3. CRIAÇÃO DE NOVAS COLUNAS E LIMPEZA (AQUI ESTÁ A CORREÇÃO!) ---

# 3.1 Limpeza Final: REMOVENDO LINHAS LIXO
# USANDO O NOME DA COLUNA LIMPA: 'nome_completo'
df_limpo = df_limpo[df_limpo['nome_completo'] != 'Colaborador'] 
df_limpo.dropna(subset=['nome_completo'], inplace=True) 
# Note que a coluna 'Colaborador' foi substituída por 'nome_completo'

# 3.2 Criação e Reordenação das Colunas
df_limpo['status_atual'] = 'Afastado' 
df_limpo['cpf'] = None
df_limpo['cargo'] = None
df_limpo['data_demissao'] = None
df_limpo['funcionario_id'] = None 

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL (dim_funcionario)
colunas_finais_sql = ['funcionario_id', 'nome_completo', 'cpf', 'data_admissao', 'cargo', 'data_demissao', 'status_atual']
df_limpo = df_limpo[colunas_finais_sql]

print("✅ Mapeamento e criação de colunas concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")
print("\n--- AMOSTRA DOS DADOS PRONTOS PARA CARGA (LOAD) ---")
print(df_limpo.head())

✅ Mapeamento e criação de colunas concluídos.
Dimensões após a limpeza: (33, 7)

--- AMOSTRA DOS DADOS PRONTOS PARA CARGA (LOAD) ---
   funcionario_id              nome_completo   cpf        data_admissao cargo  \
0            None     ADAILTON SILVA BARRETO  None  2013-10-24 00:00:00  None   
4            None  EDICLELHO ROLAND ZAMBELLI  None  2013-03-08 00:00:00  None   
10           None  RAIMUNDO DE MIRANDA SOUSA  None  2012-04-01 00:00:00  None   
13           None       REGINALDO DOS SANTOS  None  2022-02-15 00:00:00  None   
19           None     ALINE SILVA DA FONSECA  None  2022-04-19 00:00:00  None   

   data_demissao status_atual  
0           None     Afastado  
4           None     Afastado  
10          None     Afastado  
13          None     Afastado  
19          None     Afastado  


In [10]:
from sqlalchemy import create_engine
import io
from urllib.parse import quote_plus # Para lidar com caracteres especiais (@, ç)

print("\n--- FASE 3: CARGA (LOAD) - TENTATIVA FINAL DE CONEXÃO ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (TODAS AS CORREÇÕES) ---
# ------------------------------------------------------------------

DB_USER = 'postgres'
DB_HOST = '127.0.0.1'     # Usando o IP local mais seguro
DB_PORT = '5432'          # Porta correta (confirmada pelo SHOW port;)
DB_NAME = 'hserv_dw'

# Sua senha é Hserv@2025
RAW_DB_PASS = 'Hserv@2025' 

# 1. Codificando a senha (para lidar com o '@')
DB_PASS = quote_plus(RAW_DB_PASS)

# 2. Criando a string de conexão (com o encoding para lidar com o 'ç')
try:
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")

except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: Não foi possível conectar ao PostgreSQL.")
    print(f"   Por favor, verifique se a senha '{RAW_DB_PASS}' está correta.")
    print(f"   Erro original: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
# -------------------------------------------------------------------

NOME_TABELA_SQL = 'dim_funcionario'

try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento das colunas): {e}")
    raise


--- FASE 3: CARGA (LOAD) - TENTATIVA FINAL DE CONEXÃO ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 33 linhas na tabela 'dim_funcionario'...
❌ ERRO NA CARGA (Verifique o mapeamento das colunas): (psycopg2.errors.NotNullViolation) ERRO:  o valor nulo na coluna "funcionario_id" da relação "dim_funcionario" viola a restrição de não-nulo
DETAIL:  Registro que falhou contém (null, ADAILTON SILVA BARRETO, null, 2013-10-24, null, null, Afastado).

[SQL: INSERT INTO public.dim_funcionario (funcionario_id, nome_completo, cpf, data_admissao, cargo, data_demissao, status_atual) VALUES (%(funcionario_id__0)s, %(nome_completo__0)s, %(cpf__0)s, %(data_admissao__0)s, %(cargo__0)s, %(data_demissao__0)s, %(st ... 4493 characters truncated ... 2)s, %(cpf__32)s, %(data_admissao__32)s, %(cargo__32)s, %(data_demissao__32)s, %(status_atual__32)s)]
[parameters: {'cpf__0': None, 'data_admissao__0': datetime.datetime(2013, 10, 24, 0, 0), 'data_demissao__0': None,

IntegrityError: (psycopg2.errors.NotNullViolation) ERRO:  o valor nulo na coluna "funcionario_id" da relação "dim_funcionario" viola a restrição de não-nulo
DETAIL:  Registro que falhou contém (null, ADAILTON SILVA BARRETO, null, 2013-10-24, null, null, Afastado).

[SQL: INSERT INTO public.dim_funcionario (funcionario_id, nome_completo, cpf, data_admissao, cargo, data_demissao, status_atual) VALUES (%(funcionario_id__0)s, %(nome_completo__0)s, %(cpf__0)s, %(data_admissao__0)s, %(cargo__0)s, %(data_demissao__0)s, %(st ... 4493 characters truncated ... 2)s, %(cpf__32)s, %(data_admissao__32)s, %(cargo__32)s, %(data_demissao__32)s, %(status_atual__32)s)]
[parameters: {'cpf__0': None, 'data_admissao__0': datetime.datetime(2013, 10, 24, 0, 0), 'data_demissao__0': None, 'nome_completo__0': 'ADAILTON SILVA BARRETO', 'cargo__0': None, 'status_atual__0': 'Afastado', 'funcionario_id__0': None, 'cpf__1': None, 'data_admissao__1': datetime.datetime(2013, 3, 8, 0, 0), 'data_demissao__1': None, 'nome

In [11]:
import pandas as pd
from pathlib import Path
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# --- FASE 1 & 2: EXTRAÇÃO E TRANSFORMAÇÃO (E & T) ---

# --- 1. CONFIGURAÇÕES DE EXTRAÇÃO (EXTRACTION) ---
NOME_ARQUIVO = 'AFASTADOS.xlsx' 
LINHA_CABECALHO = 2 # Usando o índice 2 (3ª linha) que funcionou na leitura

# --- Mapeamento para os nomes do SQL ---
# Você já me passou que a planilha tem: 'Colaborador' e 'Data da Admissão'
COLUNAS_PARA_MANTER = {
    'Colaborador': 'nome_completo',     
    'Data da Admissão': 'data_admissao'
}

CAMINHO_BRUTO = Path('../01_raw_data') / NOME_ARQUIVO
print(f"Iniciando ETL para o arquivo: {NOME_ARQUIVO}")
print("--- FASE 1: EXTRAÇÃO (READ) ---")

try:
    df = pd.read_excel(CAMINHO_BRUTO, header=LINHA_CABECALHO)
    print(f"✅ Arquivo lido com sucesso. Dimensões brutas: {df.shape}")
except Exception as e:
    print(f"❌ ERRO NA LEITURA: {e}")
    raise

print("\n--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---")

# 2. Aplicando a renomeação
df_limpo = df.rename(columns=COLUNAS_PARA_MANTER)
colunas_finais_mapeadas = list(COLUNAS_PARA_MANTER.values())

# Selecionando APENAS as colunas mapeadas
df_limpo = df_limpo[colunas_finais_mapeadas]

# 3. CRIAÇÃO DE NOVAS COLUNAS E LIMPEZA (CORREÇÃO FINAL DE SUJEIRA E PK)

# 3.1 Correção da sujeira: Remove linhas onde a coluna 'nome_completo' ainda é o texto 'Colaborador'
# O nome da coluna limpa agora é 'nome_completo'
df_limpo = df_limpo[df_limpo['nome_completo'] != 'Colaborador'] 
df_limpo.dropna(subset=['nome_completo'], inplace=True) # Remove linhas com nome vazio

# 3.2 Criação e Padronização de Colunas Faltantes (NULLs)
df_limpo['status_atual'] = 'Afastado' # Flag criada com sucesso
df_limpo['cpf'] = None
df_limpo['cargo'] = None
df_limpo['data_demissao'] = None
# A coluna 'funcionario_id' NÃO É CRIADA, pois será gerada pelo SERIAL do PostgreSQL

# Reordenando as colunas para bater EXATAMENTE com a tabela SQL, EXCETO a coluna 'funcionario_id'
colunas_finais_sql_sem_pk = ['nome_completo', 'cpf', 'data_admissao', 'cargo', 'data_demissao', 'status_atual']
df_limpo = df_limpo[colunas_finais_sql_sem_pk]

print("✅ Mapeamento, limpeza e padronização concluídos.")
print(f"Dimensões após a limpeza: {df_limpo.shape}")


# --- FASE 3: CARGA (LOAD) ---
print("\n--- FASE 3: CARGA (LOAD) ---")

# --- 4. CONFIGURAÇÃO DA CONEXÃO COM O BANCO (TODAS AS CORREÇÕES) ---
DB_USER = 'postgres'
RAW_DB_PASS = 'Hserv@2025' # Sua senha: Hserv@2025
DB_HOST = '127.0.0.1'     # Usando o IP local mais seguro
DB_PORT = '5432'          # Porta CORRETA
DB_NAME = 'hserv_dw'

DB_PASS = quote_plus(RAW_DB_PASS) # Codifica a senha
NOME_TABELA_SQL = 'dim_funcionario'

try:
    # Adicionando o client_encoding=utf8 para resolver o erro do 'ç'
    connection_string = f"postgresql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}?client_encoding=utf8"
    engine = create_engine(connection_string)
    print("✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!")
except Exception as e:
    print(f"❌ ERRO DE CONEXÃO: {e}")
    raise

# --- 5. ENVIANDO OS DADOS PARA O POSTGRESQL ---
try:
    print(f"Iniciando a carga de {df_limpo.shape[0]} linhas na tabela '{NOME_TABELA_SQL}'...")
    
    # O df_limpo só tem as colunas secundárias. O banco de dados gera o funcionario_id.
    df_limpo.to_sql(
        NOME_TABELA_SQL,
        con=engine,
        if_exists='append', 
        index=False,
        schema='public'
    )
    
    print(f"✅ SUCESSO! Dados carregados na tabela '{NOME_TABELA_SQL}'.")

except Exception as e:
    print(f"❌ ERRO NA CARGA (Verifique o mapeamento final): {e}")
    raise

Iniciando ETL para o arquivo: AFASTADOS.xlsx
--- FASE 1: EXTRAÇÃO (READ) ---
✅ Arquivo lido com sucesso. Dimensões brutas: (126, 11)

--- FASE 2: TRANSFORMAÇÃO (CLEAN & MAP) ---
✅ Mapeamento, limpeza e padronização concluídos.
Dimensões após a limpeza: (33, 6)

--- FASE 3: CARGA (LOAD) ---
✅ Conexão com o Data Warehouse 'hserv_dw' estabelecida com sucesso!
Iniciando a carga de 33 linhas na tabela 'dim_funcionario'...
✅ SUCESSO! Dados carregados na tabela 'dim_funcionario'.
